In [11]:
from llama_index.core import Document, GPTVectorStoreIndex, ServiceContext
from llama_index.core.node_parser import SimpleNodeParser
from qdrant_client import QdrantClient
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from llama_index.vector_stores.qdrant import QdrantVectorStore

In [2]:
# Step 1: Load PDF document and split into chunks
loader = PyPDFLoader("SWE-Bench.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

In [3]:
# Step 2: Load HuggingFace BGE Embedding Model
model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

c:\Users\dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dell\.cache\huggingface\hub\models--BAAI--bge-large-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.3k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [4]:
# Step 3: Initialize Qdrant Client for the vector store
qdrant_client = QdrantClient(
    url="https://c99d7921-24d0-4759-8836-938fa2f15d91.europe-west3-0.gcp.cloud.qdrant.io:6333", 
    api_key="6cz2PvQQIfqLu2ALzGiIOCUSMBEaLV2W4MudoxYXOfywU4kt3Mu6Cw",
)


In [7]:
# Step 4: Convert text chunks into LlamaIndex Document objects
llama_documents = [Document(text=doc.page_content) for doc in texts]

In [8]:
# Step 4: Convert the texts into nodes for LlamaIndex (equivalent of documents in LangChain)
node_parser = SimpleNodeParser()
nodes = node_parser.get_nodes_from_documents(llama_documents)

In [9]:
# Step 5: Initialize the vector store with Qdrant and the HuggingFace embeddings
vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name="vector_db",  # The collection name in Qdrant
    embedding_model=embeddings  # Use the HuggingFace BGE embeddings
)

In [13]:
# Step 6: Build the LlamaIndex with the Qdrant vector store
index = GPTVectorStoreIndex(
    nodes=nodes,
    vector_store=vector_store,
    embed_model=embeddings  # Explicitly pass the HuggingFace embeddings model
)

In [30]:
# Step 7: Query the index without using an LLM
query_engine = index.as_query_engine(llm=None)
response = query_engine.query("What is the SWE-Bench benchmark?")
print(response)

ValueError: 
******
Could not load OpenAI model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

To disable the LLM entirely, set llm=None.
******